In [11]:
# ==============================================================================
# CELLULE 1 : IMPORTS & CONFIGURATION (ULTIMATE EDITION)
# ==============================================================================
import pandas as pd
import numpy as np
import joblib
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss
from sklearn.linear_model import LogisticRegression

# Optimization
import optuna

sys.path.append('..')
# On suppose que le module src est disponible comme dans le projet original
from src.sampling import get_sample_weights

import warnings
warnings.filterwarnings('ignore')

print("✅ Environnement ULTIMATE prêt (XGBoost + LightGBM + CatBoost + Optuna).")

✅ Environnement ULTIMATE prêt (XGBoost + LightGBM + CatBoost + Optuna).


In [12]:
# ==============================================================================
# CELLULE 2 : CHARGEMENT & META-LABELING AVANCÉ
# ==============================================================================
print("\n1. Chargement et Meta-Labeling Avancé...")

# 1. CHARGEMENT DES DONNÉES (À faire en premier !)
X = pd.read_parquet('../data/processed/train_stationary.parquet')

# Récupération de la target brute si absente
if 'forward_returns' not in X.columns:
    raw = pd.read_csv('../data/raw/train.csv').set_index('date_id')
    X['forward_returns'] = raw['forward_returns']

# 2. FEATURE ENGINEERING (Maintenant que X est chargé)
# --- AMÉLIORATION : FEATURE ENGINEERING ---
# Création d'interactions pour aider les modèles d'arbres
# Ratio Signal/Bruit : Momentum divisé par la Volatilité
X['M1_div_V3'] = X['M1'] / (X['V3'] + 1e-6)
X['M6_div_V13'] = X['M6'] / (X['V13'] + 1e-6)

# Interactions non-linéaires
X['M1_x_V3'] = X['M1'] * X['V3']
X['Vol_Regime'] = (X['V3'] > X['V3'].rolling(20).mean()).astype(int) # 1 si volatilité en hausse

# Mettez à jour la liste des features à garder plus tard (Cellule 3)
# Ajoutez ces nouvelles colonnes à votre liste TOP_FEATURES ou relancez le MDA.

# 3. SIGNAL PRIMAIRE
# --- AMÉLIORATION 1 : SIGNAL PRIMAIRE PLUS ROBUSTE ---
# Au lieu de juste M1, on utilise un "Consensus de Momentum"
primary_signal = pd.Series(0, index=X.index)
primary_signal.loc[(X['M1'] > 0) & (X['M6'] > 0)] = 1  # Strong Buy
primary_signal.loc[(X['M1'] < 0) & (X['M6'] < 0)] = -1 # Strong Sell

# 4. META-LABELING
# --- AMÉLIORATION : CIBLE DYNAMIQUE (TRIPLE BARRIER SIMPLIFIÉE) ---
# On utilise la volatilité (V3) comme seuil dynamique.
daily_vol = X['V3']
threshold = 0.5 * daily_vol # Barrière horizontale dynamique

# Gagnant si : (Signal Achat ET Hausse > Seuil) OU (Signal Vente ET Baisse < -Seuil)
y_meta = ((primary_signal == 1) & (X['forward_returns'] > threshold)) | \
         ((primary_signal == -1) & (X['forward_returns'] < -threshold))
y_meta = y_meta.astype(int)

# 5. FILTRAGE DES ÉVÉNEMENTS (Création explicite des variables manquantes)
mask_events = primary_signal != 0
X_events = X[mask_events].copy()
y_events = y_meta[mask_events].copy()

print(f"Signal Primaire Activé : {mask_events.sum()} fois sur {len(X)}")
print(f"Distribution des Meta-Labels (Trades gagnants) :\n{y_events.value_counts()}")

# 6. CALCUL DES POIDS
# --- CALCUL DES POIDS ---
price_proxy = (1 + X_events['forward_returns']).cumprod()
w_raw = (np.log(price_proxy).diff().shift(-1).abs() * 100).fillna(0)
w_train = w_raw / w_raw.mean()

# 7. NETTOYAGE ET SPLIT
# --- NETTOYAGE ---
cols_to_drop = ['forward_returns', 'market_forward_excess_returns', 
                'lagged_forward_returns', 'lagged_market_forward_excess_returns', 
                'risk_free_rate', 'date_id']
X_clean = X_events.drop(columns=[c for c in cols_to_drop if c in X_events.columns])

# Split Temporel (80/20)
split = int(len(X_clean) * 0.80)
X_tr, X_val = X_clean.iloc[:split], X_clean.iloc[split:]
y_tr, y_val = y_events.iloc[:split], y_events.iloc[split:]
w_tr, w_val = w_train.iloc[:split], w_train.iloc[split:]

print(f"Train set: {X_tr.shape}, Val set: {X_val.shape}")


1. Chargement et Meta-Labeling Avancé...
Signal Primaire Activé : 1245 fois sur 2052
Distribution des Meta-Labels (Trades gagnants) :
0    1232
1      13
Name: count, dtype: int64
Train set: (996, 101), Val set: (249, 101)


In [13]:
# ==============================================================================
# CELLULE 3 : FEATURE SELECTION (MDA)
# ==============================================================================
# (On garde le MDA du notebook précédent car c'est déjà excellent)
# Pour gagner du temps d'exécution ici, on simule une sélection déjà faite.
TOP_FEATURES = ['M1', 'M6', 'S1', 'V3', 'S3', 'S10', 'M15', 'P6', 'M16', 'M5', 
                'S5', 'V4', 'P10_ffd_vol', 'E2', 'M2', 'I4', 'E12', 'P8']
X_tr_lean = X_tr[TOP_FEATURES]
X_val_lean = X_val[TOP_FEATURES]
print(f"\n2. Features sélectionnées : {len(TOP_FEATURES)}")


2. Features sélectionnées : 18


[I 2025-11-29 17:11:56,112] A new study created in memory with name: no-name-ecfe2952-73f7-4667-b594-06704761a140



3. Optimisation Bayésienne (Version Robuste)...


[W 2025-11-29 17:11:56,423] Trial 0 failed with parameters: {'classifier': 'XGB', 'n_estimators': 421, 'max_depth': 5, 'learning_rate': 0.020478033281067072, 'subsample': 0.9460772141124152} because of the following error: The value nan is not acceptable.
[W 2025-11-29 17:11:56,423] Trial 0 failed with value nan.
[W 2025-11-29 17:11:56,587] Trial 1 failed with parameters: {'classifier': 'LGBM', 'n_estimators': 135, 'max_depth': 5, 'learning_rate': 0.03366197154148903, 'num_leaves': 25} because of the following error: The value nan is not acceptable.
[W 2025-11-29 17:11:56,588] Trial 1 failed with value nan.
[W 2025-11-29 17:11:56,794] Trial 2 failed with parameters: {'classifier': 'LGBM', 'n_estimators': 469, 'max_depth': 3, 'learning_rate': 0.08444257872162673, 'num_leaves': 24} because of the following error: The value nan is not acceptable.
[W 2025-11-29 17:11:56,795] Trial 2 failed with value nan.
[W 2025-11-29 17:11:56,919] Trial 3 failed with parameters: {'classifier': 'LGBM', 'n

ValueError: No trials are completed yet.

In [ ]:
# ==============================================================================
# CELLULE 5 : ENTRAÎNEMENT DE L'ENSEMBLE, CALIBRATION & STACKING
# ==============================================================================
print("\n4. Entraînement de l'Ensemble (XGB + LGBM + CatBoost), Calibration & Stacking...")

from sklearn.linear_model import LogisticRegression

# --- 1. CONFIGURATION & ENTRAÎNEMENT DES MODÈLES DE BASE ---

# A. XGBoost (avec params Optuna ou défaut robustes)
xgb_params = {k: v for k, v in study.best_params.items() if k != 'classifier'} if study.best_params.get('classifier') == 'XGB' else {}
model_xgb = XGBClassifier(**xgb_params, n_jobs=-1, random_state=42)

# B. LightGBM
lgb_params = {k: v for k, v in study.best_params.items() if k != 'classifier'} if study.best_params.get('classifier') == 'LGBM' else {}
model_lgb = LGBMClassifier(**lgb_params, n_jobs=-1, random_state=42, verbose=-1)

# C. CatBoost (souvent excellent 'out of the box')
model_cat = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.03, verbose=0, random_state=42)

# Entraînement sur le TRAIN SET (avec Sample Weights)
print("   - Training XGB...")
model_xgb.fit(X_tr_lean, y_tr, sample_weight=w_tr.values)

print("   - Training LGBM...")
model_lgb.fit(X_tr_lean, y_tr, sample_weight=w_tr.values)

print("   - Training CatBoost...")
model_cat.fit(X_tr_lean, y_tr, sample_weight=w_tr.values)


# --- 2. CALIBRATION DES PROBABILITÉS ---
# On utilise le set de VALIDATION pour calibrer les scores bruts
print("   - Calibration des probabilités...")

# 'cv="prefit"' car les modèles sont déjà entraînés ci-dessus
calib_xgb = CalibratedClassifierCV(model_xgb, cv='prefit', method='isotonic')
calib_lgb = CalibratedClassifierCV(model_lgb, cv='prefit', method='isotonic')
calib_cat = CalibratedClassifierCV(model_cat, cv='prefit', method='isotonic')

# Fit de la calibration sur le set de validation
calib_xgb.fit(X_val_lean, y_val)
calib_lgb.fit(X_val_lean, y_val)
calib_cat.fit(X_val_lean, y_val)


# --- 3. AMÉLIORATION : STACKING (Méta-Modèle) ---
print("   - Entraînement du Stacker (Logistic Regression)...")

# A. Générer les prédictions (probas) des 3 modèles calibrés sur le set de VALIDATION
# (Ces prédictions deviennent les "features" pour le méta-modèle)
meta_features_val = pd.DataFrame({
    'xgb': calib_xgb.predict_proba(X_val_lean)[:, 1],
    'lgb': calib_lgb.predict_proba(X_val_lean)[:, 1],
    'cat': calib_cat.predict_proba(X_val_lean)[:, 1]
})

# B. Entraîner le Meta-Modèle (Stacker)
# Il apprendra à pondérer XGB, LGB et Cat pour maximiser le score final.
stacker = LogisticRegression()
stacker.fit(meta_features_val, y_val)

# Affichage des coefficients (Poids) accordés à chaque modèle
print(f"⚖️  Poids du Stacking : XGB={stacker.coef_[0][0]:.2f}, LGB={stacker.coef_[0][1]:.2f}, Cat={stacker.coef_[0][2]:.2f}")
# Note : Des poids négatifs ou nuls peuvent apparaître si les modèles sont très corrélés.

# C. Sauvegarder le Stacker et les modèles
if not os.path.exists('../submission'):
    os.makedirs('../submission')

joblib.dump(stacker, '../submission/model_stacker.pkl')
# On sauvegarde aussi les modèles calibrés pour pouvoir générer les inputs du stacker en prod
joblib.dump(calib_xgb, '../submission/model_xgb.pkl')
joblib.dump(calib_lgb, '../submission/model_lgb.pkl')
joblib.dump(calib_cat, '../submission/model_cat.pkl')

print("✅ Modèles et Stacker sauvegardés.")


4. Entraînement de l'Ensemble (XGB + LGBM + CatBoost) & Calibration...
   - Training XGB...
   - Training LGBM...
   - Training CatBoost...
   - Calibration des probabilités...


,estimator,<catboost.cor...001CBFC2F83E0>
,method,'isotonic'
,cv,'prefit'
,n_jobs,None
,ensemble,'auto'


In [ ]:
# ==============================================================================
# CELLULE 6 : BET SIZING & EXPORT
# ==============================================================================
print("\n5. Logique de Trading Finale (Bet Sizing)...")

def bet_sizing(probability):
    """
    Transforme une probabilité (0-1) en taille de position (0-1).
    Utilise une fonction sigmoïde pour être agressif seulement si la confiance est haute.
    """
    # Centré sur 0.6 (on ne parie que si proba > 0.5)
    # Le facteur 10 contrôle la pentitude
    if probability <= 0.5:
        return 0.0
    
    # Formule Sigmoïde modifiée
    size = 2 * (1 / (1 + np.exp(-10 * (probability - 0.5))) - 0.5)
    return np.clip(size, 0, 1)

# Test sur quelques valeurs
test_probs = [0.5, 0.55, 0.6, 0.7, 0.9]
print("Test Bet Sizing :")
for p in test_probs:
    print(f"  Proba {p:.2f} -> Position {bet_sizing(p):.2f}")

# --- SAUVEGARDE ---
if not os.path.exists('../submission'):
    os.makedirs('../submission')

# On sauvegarde les modèles calibrés
joblib.dump(calib_xgb, '../submission/model_xgb.pkl')
joblib.dump(calib_lgb, '../submission/model_lgb.pkl')
# Catboost peut nécessiter sa propre méthode de save ou pickle
joblib.dump(calib_cat, '../submission/model_cat.pkl')
joblib.dump(TOP_FEATURES, '../submission/features_list.pkl')

print(f"\n💾 TOUT EST SAUVEGARDÉ DANS ../submission/ !")
print("   Modèle prêt pour l'inférence (Ensemble Moyenné + Bet Sizing).")


5. Logique de Trading Finale (Bet Sizing)...
Test Bet Sizing :
  Proba 0.50 -> Position 0.00
  Proba 0.55 -> Position 0.24
  Proba 0.60 -> Position 0.46
  Proba 0.70 -> Position 0.76
  Proba 0.90 -> Position 0.96

💾 TOUT EST SAUVEGARDÉ DANS ../submission/ !
   Modèle prêt pour l'inférence (Ensemble Moyenné + Bet Sizing).
